In [12]:
import pandas as pd
import cv2
import numpy as np
from pathlib import Path

from torch.utils.data import Dataset, DataLoader

class CXRDatasetrFull(Dataset):
    def __init__(self, root, csv_path='train.csv'):
        '''
        param: csv_path: 
        param: meta_path: initial resolutions
        '''
        self.root = Path(root)
        self.df = pd.read_csv(csv_path)
        self.ids = self.df.image_id.unique()

    def __len__(self, ):
        return len(self.ids)

    def __getitem__(self, idx):
        name = self.ids[idx]
        tmp = self.df[self.df.image_id == name].copy()
        path = self.root / f'{name}.npy'
        img = np.load(str(path))

        # rescale boxes

        tmp = tmp.sort_values(['class_name', 'rad_id'])

        meta = tmp[['class_name', 'class_id', 'rad_id']].to_dict('list')
        bboxes = tmp[['x_min', 'y_min', 'x_max', 'y_max']].values.astype(np.uint16)

        return {'image': img, **meta, 'bboxes': bboxes}

    def get_by_name(self, name):
        return self.__getitem__(list(self.ids).index(name))

if __name__ == '__main__':
    csv_path = '/home/semyon/data/VinBigData/train.csv'
    root = '/home/semyon/data/VinBigData/train/'
    d = CXRDatasetrFull(root, csv_path)
    entry = d[2]
    img = entry['image']
    rad_ids = entry['rad_id']
    class_names = entry['class_name']
    class_id = entry['class_id']
    bboxes = entry['bboxes']
    print(class_names)

['Aortic enlargement', 'Cardiomegaly', 'Cardiomegaly', 'Cardiomegaly', 'Pleural effusion', 'Pleural thickening']


In [13]:
img.shape

(2336, 2080)